# Automated ML
## Introduction

This notebook is automatically generated by the Fabric low-code AutoML wizard based on your selections. Whether you're building a regression model, a classifier, or another machine-learning solution, this tool simplifies the process by transforming your goals into executable code. You can easily modify any settings or code snippets to better align with your requirements.

### What is FLAML?

[FLAML (Fast and Lightweight Automated Machine Learning)](https://aka.ms/fabric-automl) is an open-source AutoML library designed to quickly and efficiently find the best machine learning models and hyperparameters. FLAML optimizes for speed, accuracy, and cost, making it an excellent choice for a wide range of machine-learning tasks.

### Steps in this notebook

1. **Load the data**: Import your dataset.
2. **Generate features**: Automatically transform and preprocess your data to improve model performance.
3. **Use AutoML to find your best model**: Use FLAML to automatically select the most suitable model and optimize its parameters.
4. **Save the final machine learning model**: Store the trained model for future use.
5. **Generate predictions**: Use the saved model to predict outcomes on new data.

> [!IMPORTANT]
> **The forecasting functionality is currently supported only on Pandas DataFrames.**
> **Automated ML is currently supported on Fabric Runtimes 1.2+ or any Fabric environment with Spark 3.4+.**


In [11]:
%pip install scikit-learn==1.5.1


StatementMeta(, 4199b5b7-d4c5-4775-9913-fb81b5ba8c31, 22, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



### Default notebook optimization

This cell configures the logging and warning settings to reduce unnecessary output and focus on critical information. It suppresses specific warnings and logs from the underlying libraries, ensuring a cleaner and more readable notebook experience.

In [12]:
import logging
import warnings
 
logging.getLogger('synapse.ml').setLevel(logging.CRITICAL)
logging.getLogger('mlflow.utils').setLevel(logging.CRITICAL)
warnings.simplefilter('ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=UserWarning)

StatementMeta(, 4199b5b7-d4c5-4775-9913-fb81b5ba8c31, 24, Finished, Available, Finished)

## Step 1: Load the Data

This cell is responsible for importing the raw data from the specified source into the notebook environment. The data could come from various sources, such as a file or table in your lakehouse.

Once loaded, this data will serve as the input for subsequent steps, such as data transformation, model training, and evaluation.

In [13]:
import re
import pandas as pd
import numpy as np

df = spark.read.format("delta").load(
    "Tables/forecastqty_customdata_v2_na"
).cache()
# Transform to pandas according to the selected models
X = df.limit(100000).toPandas() # Use df.toPandas() to use all the data
X = X.rename(columns = lambda c:re.sub('[^A-Za-z0-9_]+', '_', c))  # Replace not supported characters in column name with underscore to avoid invalid character for model training and saving

target_col = re.sub('[^A-Za-z0-9_]+', '_', "Quantity_Invoiced")


StatementMeta(, 4199b5b7-d4c5-4775-9913-fb81b5ba8c31, 25, Finished, Available, Finished)

In [14]:
display(X)

StatementMeta(, 4199b5b7-d4c5-4775-9913-fb81b5ba8c31, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ab2cbeec-5a3d-4eef-b7e9-0073d30c6d89)

## Step 2: Generate features

Featurization is the process of transforming raw data into a format optimized for training a machine learning model. It ensures the model can access the most relevant information, significantly impacting its accuracy and performance.

This step applies various techniques to refine the data, enhance its quality, and make it compatible with the selected algorithms, helping the model learn patterns more effectively.

In [15]:
# Set Functions if needed for Featurization
def create_fillna_processor(
    df, mean_features=None, median_features=None, mode_features=None
):
    """
    Create a ColumnTransformer that fills missing values in a DataFrame using different strategies
    based on the skewness of the numerical features and the specified feature lists.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    mean_features (list, optional): List of features to impute using the mean strategy. Defaults to None.
    median_features (list, optional): List of features to impute using the median strategy. Defaults to None.
    mode_features (list, optional): List of features to impute using the mode strategy. Defaults to None.

    Returns:
    ColumnTransformer: A fitted ColumnTransformer that can be used to transform the DataFrame.
    list: List of all features supported by SimpleImputer in the DataFrame.
    list: List of datetime features in the DataFrame.
    """
    if mean_features is None:
        mean_features = []
    if median_features is None:
        median_features = []
    if mode_features is None:
        mode_features = []
    all_features = mean_features + median_features + mode_features
    # Group features by their imputation needs
    mean_features = [
        col
        for col in df.select_dtypes(include=["number"]).columns
        if df[col].skew(skipna=True) <= 1 and col not in all_features
    ] + mean_features
    median_features = [
        col
        for col in df.select_dtypes(include=["number"]).columns
        if df[col].skew(skipna=True) > 1 and col not in all_features
    ] + median_features
    all_features = mean_features + median_features
    datetime_features = df.select_dtypes(include=["datetime"]).columns.tolist()
    mode_features = [col for col in df.columns.tolist() if col not in all_features + datetime_features]

    transformers = []

    if mean_features:
        transformers.append(
            ("mean_imputer", SimpleImputer(strategy="mean"), mean_features)
        )
    if median_features:
        transformers.append(
            ("median_imputer", SimpleImputer(strategy="median"), median_features)
        )
    if mode_features:
        transformers.append(
            ("mode_imputer", SimpleImputer(strategy="most_frequent"), mode_features)
        )

    column_transformer = ColumnTransformer(transformers=transformers)
    all_features = mean_features + median_features + mode_features

    return column_transformer.fit(df), all_features, datetime_features


def fillna(df, processor, all_features, datetime_features):
    """
    Fill missing values in a DataFrame using a specified processor and mode imputation.

    Parameters:
    df (pd.DataFrame): The input DataFrame with missing values.
    processor (object): An object with a `transform` method that processes the DataFrame.
    all_features (list): List of all features supported by SimpleImputer in the DataFrame.
    datetime_features (list): List of datetime features in the DataFrame.

    Returns:
    pd.DataFrame: A DataFrame with missing values filled.
    """
    filled_array = processor.transform(df)
    filled_df = pd.DataFrame(filled_array, columns=all_features)
    if datetime_features:
        datetime_data = df[datetime_features]
        datetime_data.ffill()
        filled_df = pd.concat([datetime_data, filled_df], axis=1)
    for col in df.columns:
        filled_df[col].fillna(filled_df[col].mode()[0], inplace=True)

    return filled_df


StatementMeta(, 4199b5b7-d4c5-4775-9913-fb81b5ba8c31, 27, Finished, Available, Finished)

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

time_col = "Full_Date"
ts_col = X.pop(time_col)
X.insert(0, time_col, ts_col.apply(lambda x: np.datetime64(x, "ns")))

# Preserve important ID columns for frequency filling
id_columns = ["ParentAccountName", "RiekeUniversalProductCode", "Full_Date"]
X_id = X[id_columns].copy()

# Clean and select features
X = X.convert_dtypes()
X = X.dropna(axis=1, how='all')
X_features = X.select_dtypes(include=['number', 'datetime', 'category'])

# Reattach preserved ID columns
X = pd.concat([X_id, X_features.drop(columns=[col for col in X_id.columns if col in X_features.columns])], axis=1)

from sklearn.model_selection import train_test_split

# Split before reindexing/frequency enforcement
X_train, X_test = train_test_split(X, test_size=int(X.shape[0] / 12 * 0.2) * 12, shuffle=False, random_state=41)

# DO NOT drop duplicates twice — keep it only here
X_train = X_train.drop_duplicates(subset=["ParentAccountName", "RiekeUniversalProductCode", "Full_Date"])

# Enforce regular monthly frequency per Customer/Product
full_dates = pd.date_range(start=X_train["Full_Date"].min(), end=X_train["Full_Date"].max(), freq="MS")
group_cols = ["ParentAccountName", "RiekeUniversalProductCode"]
full_index = pd.MultiIndex.from_product([
    X_train[group_cols[0]].dropna().unique(),
    X_train[group_cols[1]].dropna().unique(),
    full_dates
], names=[group_cols[0], group_cols[1], "Full_Date"])

# Reindex to force regular frequency
X_train.set_index(group_cols + ["Full_Date"], inplace=True)
X_train = X_train.reindex(full_index).reset_index()

# Fill missing Quantity with 0s (safe assumption)
X_train["Quantity_Invoiced"].fillna(0, inplace=True)

# Final cleanup
X_train = X_train.dropna(subset=["Full_Date"])

# Impute the rest
mean_features, median_features, mode_features = [], [], []
preprocessor, all_features, datetime_features = create_fillna_processor(X_train, mean_features, median_features, mode_features)
X_train = fillna(X_train, preprocessor, all_features, datetime_features)
X_test = fillna(X_test, preprocessor, all_features, datetime_features)

# REMOVE this — no need to drop Full_Date duplicates again:
# X_train = X_train.drop_duplicates(subset=["Full_Date"])

# Now get target columns
y_train = X_train.pop(target_col)
y_test = X_test.pop(target_col)

display(X_train[:10])

StatementMeta(, 4199b5b7-d4c5-4775-9913-fb81b5ba8c31, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4961b5c4-e5ca-4626-a39d-32b2807c6370)

## Step 3: Use AutoML to find your best model

We will now use FLAML's AutoML to automatically find the best machine learning model for our data. AutoML (Automated Machine Learning) simplifies the model selection process by automatically testing and tuning various algorithms and configurations, helping us quickly identify the most effective model with minimal manual effort.

### Tracking results with experiments in Fabric

Experiments in Fabric let you track the results of your AutoML process, providing a comprehensive view of all the metrics and parameters from your trials.

In [17]:
# MLFlow Logging Related

import mlflow

mlflow.autolog(exclusive=False)
mlflow.set_experiment("ForecastQty_CustomerProductQty_Forecast_v1_NA")


StatementMeta(, 4199b5b7-d4c5-4775-9913-fb81b5ba8c31, 29, Finished, Available, Finished)

2025/07/08 17:10:11 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


<Experiment: artifact_location='', creation_time=1751747605138, experiment_id='0e9dc7bc-42c8-412d-b69d-26cbed06ac91', last_update_time=None, lifecycle_stage='active', name='ForecastQty_CustomerProductQty_Forecast_v1_NA', tags={}>

#### Configure the AutoML trial and settings

These configurations are driven by the AutoML mode and task selected in the wizard. For example, if you select "quick prototype", you'll see a setting for time budget.

In [18]:
# Import the AutoML class from the FLAML package
import flaml
from flaml import AutoML

# Define AutoML settings
settings = {
    "time_budget": 3600, # Total running time in seconds
    "estimator_list": ['lgbm', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'prophet'],  # estimator_list for spark35 forecasting 
    "task": "ts_forecast",  # Task type 
    "log_file_name": "flaml_experiment.log",  # FLAML log file
    "seed": 41 , # Random seed 
    "mlflow_exp_name": "ForecastQty_CustomerProductQty_Forecast_v1_NA",  # MLflow experiment name
    "use_spark": True, # whether to use Spark for distributed training
    "n_concurrent_trials": 3,  # the maximum number of concurrent trials 
    "verbose": 1, 
    "featurization": "auto", 
}

if flaml.__version__ > "2.3.3":
    settings["entrypoint"] = "low-code"

# Create an AutoML instance
automl = AutoML(**settings)


StatementMeta(, 4199b5b7-d4c5-4775-9913-fb81b5ba8c31, 30, Finished, Available, Finished)

2025/07/08 17:10:12 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.
2025/07/08 17:10:13 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2025/07/08 17:10:13 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/07/08 17:10:16 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.
2025/07/08 17:10:17 INFO mlflow.tracking.fluent: Autologging successfully enabled for pytorch_lightning.


#### Run the AutoML trial

Run the AutoML trial, with all trials being tracked as experiment runs. The trial is performed on the processed dataset, using the `Exited` variable as the target, and applying the defined configurations for optimal model selection.

In [19]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Convert date column
time_col = "Full_Date"
X[time_col] = pd.to_datetime(X[time_col])
X.insert(0, time_col, X.pop(time_col))

# Keep important ID columns
group_cols = ["ParentAccountName", "RiekeUniversalProductCode"]
id_cols = group_cols + [time_col]
X_id = X[id_cols].copy()

# Clean features
X = X.convert_dtypes()
X = X.dropna(axis=1, how="all")
X_features = X.select_dtypes(include=["number", "datetime", "category"])
X = pd.concat([X_id, X_features.drop(columns=[c for c in id_cols if c in X_features.columns])], axis=1)

# Remove duplicates
X = X.drop_duplicates()

# Aggregate duplicate group/date rows
X = X.groupby(group_cols + [time_col], as_index=False).agg({"Quantity_Invoiced": "sum"})

# Train/test split (keep time order)
X_train, X_test = train_test_split(
    X, test_size=int(X.shape[0] / 12 * 0.2) * 12, shuffle=False, random_state=41
)

# Fill missing months
full_dates = pd.date_range(start=X_train[time_col].min(), end=X_train[time_col].max(), freq="MS")
full_index = pd.MultiIndex.from_product(
    [X_train[group_cols[0]].unique(), X_train[group_cols[1]].unique(), full_dates],
    names=group_cols + [time_col]
)
X_train = X_train.set_index(group_cols + [time_col]).reindex(full_index).reset_index()

# Fill missing values
X_train["Quantity_Invoiced"] = X_train["Quantity_Invoiced"].fillna(0)

# Remove bad rows
X_train = X_train.dropna(subset=[time_col])

# Drop lag columns and fix types
X_train = X_train[[col for col in X_train.columns if not col.startswith("lag_")]]
for col in X_train.columns:
    if X_train[col].dtype == "object":
        try:
            X_train[col] = pd.to_numeric(X_train[col])
        except Exception:
            print(f"Dropping column: {col}")
            X_train.drop(columns=[col], inplace=True)

# Final deduplication
X_train = X_train.drop_duplicates()
duplicates = X_train.duplicated(subset=[time_col])
if duplicates.any():
    print("Dropping duplicate timestamps")
    X_train = X_train[~duplicates]
else:
    print("No duplicate timestamps")

# Target column
target_col = "Quantity_Invoiced"
y_train = X_train.pop(target_col)

StatementMeta(, 4199b5b7-d4c5-4775-9913-fb81b5ba8c31, 31, Finished, Available, Finished)

Dropping column: ParentAccountName
Dropping column: RiekeUniversalProductCode
Dropping duplicate timestamps


In [20]:
with mlflow.start_run(nested=True, run_name="ForecastQty_CustomerProductQty_Forecast_Model_v1_NA"):
    automl.fit(
        X_train=X_train, 
        y_train=y_train,  # target column of the training data 
        period=12, 
    )

StatementMeta(, 4199b5b7-d4c5-4775-9913-fb81b5ba8c31, 32, Finished, Available, Finished)

INFO:flaml.automl.task.time_series_task:Couldn't import orbit, skipping
[I 2025-07-08 17:10:22,201] A new study created in memory with name: optuna


[I 2025-07-08 17:10:38,817] A new study created in memory with name: optuna


2025/07/08 18:49:57 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2025/07/08 18:49:57 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/08 18:49:57 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.
2025/07/08 18:49:57 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/07/08 18:49:57 INFO mlflow.tracking.fluent: Autologging successfully enabled for pytorch_lightning.
2025/07/08 18:49:57 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.


## Step 4: Save the final machine learning model

Upon completing the AutoML trial, you can now save the final, tuned model as an ML model in Fabric.

In [21]:
model_path = f"runs:/{automl.best_run_id}/model"

# Register the model to the MLflow registry
registered_model = mlflow.register_model(model_uri=model_path, name="ForecastQty_CustomerProductQty_Forecast_Model_v1_NA")

# Print the registered model's name and version
print(f"Model '{registered_model.name}' version {registered_model.version} registered successfully.")

StatementMeta(, 4199b5b7-d4c5-4775-9913-fb81b5ba8c31, 33, Finished, Available, Finished)

Successfully registered model 'ForecastQty_CustomerProductQty_Forecast_Model_v1_NA'.
Created version '1' of model 'ForecastQty_CustomerProductQty_Forecast_Model_v1_NA'.


## Step 5: Generate predictions

1. Generate predictions.

In [22]:
loaded_model_pred = automl.predict(X_test)
print('Predicted labels', loaded_model_pred)


StatementMeta(, 4199b5b7-d4c5-4775-9913-fb81b5ba8c31, 34, Finished, Available, Finished)

Predicted labels 1906   -0.000161
1907   -0.000089
1908   -0.001552
1909   -0.000870
1910   -0.000214
          ...   
2369    0.001498
2370   -0.000246
2371    0.000040
2372    0.006959
2373    0.011507
Name: Quantity_Invoiced, Length: 468, dtype: float64


2. Save the predictions to a table.

In [23]:
from pyspark.sql.types import FloatType
predictions = spark.createDataFrame(loaded_model_pred, FloatType())
saved_name = "forecastqty_customdata_v2_na_predictions".replace(".", "_")
predictions.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(f"Tables/{saved_name}")

StatementMeta(, 4199b5b7-d4c5-4775-9913-fb81b5ba8c31, 35, Finished, Available, Finished)